In [1]:
!pip install Keras==2.0.0

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7812459001613525486, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11297803469
 locality {
   bus_id: 1
 }
 incarnation: 4506383088414905741
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [3]:
import os
import numpy as np
import warnings
import keras
from keras.datasets import cifar10

from keras.models import Sequential
from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.engine.topology import get_source_inputs

Using TensorFlow backend.


In [0]:
batch_size = 30
num_classes = 10
epochs = 10
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[1:])
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (50000, 32, 32, 3)
(32, 32, 3)
50000 train samples
10000 test samples


Sequentialで繋いで書くこともできるんだけども、複雑な構造になった時にコードがダサくなる。

In [6]:

model = Sequential()
# Block 1
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', input_shape=x_train.shape[1:]))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))
# Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))
# Block 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))
# Block 4
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))
# Block 5
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))
# Classification block
model.add(Flatten(name='flatten'))
model.add(Dense(4096, activation='relu', name='fc1'))
model.add(Dense(4096, activation='relu', name='fc2'))
model.add(Dense(num_classes, activation='softmax', name='predictions'))

opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# model.train_on_batch(x_train, y_train)

model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
 6180/50000 [==>...........................] - ETA: 188s - loss: 2.1418 - acc: 0.1693

49980/50000 [============================>.] - ETA: 0s - loss: 1.7439 - acc: 0.3248

50000/50000 [==============================] - 213s - loss: 1.7438 - acc: 0.3249 - val_loss: 1.4599 - val_acc: 0.4758
Epoch 2/100
 9280/50000 [====>.........................] - ETA: 166s - loss: 1.3753 - acc: 0.4940

49980/50000 [============================>.] - ETA: 0s - loss: 1.2437 - acc: 0.5557

50000/50000 [==============================] - 212s - loss: 1.2438 - acc: 0.5557 - val_loss: 1.0768 - val_acc: 0.6196
Epoch 3/100
 9280/50000 [====>.........................] - ETA: 166s - loss: 1.0242 - acc: 0.6416

...

50000/50000 [==============================] - 211s - loss: 0.9126 - acc: 0.7293 - val_loss: 1.0485 - val_acc: 0.6608
Epoch 9/100
 9280/50000 [====>.........................] - ETA: 166s - loss: 0.8377 - acc: 0.7391

49980/50000 [============================>.] - ETA: 0s - loss: 0.8959 - acc: 0.7262

50000/50000 [==============================] - 211s - loss: 0.8958 - acc: 0.7262 - val_loss: 1.4993 - val_acc: 0.6854
Epoch 10/100
 9240/50000 [====>.........................] - ETA: 166s - loss: 0.9203 - acc: 0.7255

18260/50000 [=========>....................] - ETA: 129s - loss: 0.9996 - acc: 0.7302

KeyboardInterrupt: ignored

In [0]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


# 違う書き方で書く

In [15]:
img_input = Input(shape=x_train.shape[1:])
print(img_input.shape)
# Block 1
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
# Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
# Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
# Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
# Classification block
x = Flatten(name='flatten')(x)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(num_classes, activation='softmax', name='predictions')(x)

model = Model(img_input, x, name='resnet50')

(?, 32, 32, 3)


In [0]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
13580/50000 [=======>......................] - ETA: 150s - loss: 2.3027 - acc: 0.0977

49980/50000 [============================>.] - ETA: 0s - loss: 2.3027 - acc: 0.0977

50000/50000 [==============================] - 213s - loss: 2.3027 - acc: 0.0977 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 2/100
 9280/50000 [====>.........................] - ETA: 167s - loss: 2.3027 - acc: 0.0992

49980/50000 [============================>.] - ETA: 0s - loss: 2.3027 - acc: 0.1000

50000/50000 [==============================] - 212s - loss: 2.3027 - acc: 0.1000 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 3/100
 9280/50000 [====>.........................] - ETA: 166s - loss: 2.3026 - acc: 0.1012

49980/50000 [============================>.] - ETA: 0s - loss: 2.3026 - acc: 0.0991

50000/50000 [==============================] - 212s - loss: 2.3026 - acc: 0.0991 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 4/100
 5360/50000 [==>...........................] - ETA: 183s - loss: 2.3026 - acc: 0.0974